## Reconstruction with four DFT calculations as initializations

In [ ]:
import warnings as wn
wn.filterwarnings("ignore")

import os
import numpy as np
import fuller
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpes import fprocessing as fp, analysis as aly
from scipy import io as sio
%matplotlib inline

In [ ]:
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = 'Arial'
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

# Create plot folder if needed
if not os.path.exists('../results/figures'):
    os.mkdir('../results/figures')

In [ ]:
# Load preprocessed photoemission data
bcsm = np.load(r'../data/processed/hslines/WSe2_vcut.npy')
Evals = fp.readBinnedhdf5(r'../data/pes/3_smooth.h5')['E']
ehi, elo = Evals[0], Evals[469]

### Main Figure 2a

In [ ]:
# Specify the band shift hyperparameters (including the global shift of 0.65)
dftbands = sio.loadmat('../data/theory/hslines/initials_DFT_G-M.mat')    
lda_th_shift = dftbands['LDA'][100:125, 0].max()
pbe_th_shift = dftbands['PBE'][100:125, 0].max()
pbesol_th_shift = dftbands['PBEsol'][100:125, 0].max()
hse_th_shift = dftbands['HSE'][100:125, 0].max()
eshift = 0.65

lda_shift = np.array([0.4, 0.2, 0.3, 0.25, 0.22, 0.3, 0.3, 0.25, 0.2, 0.32, 0.4, 0.4, 0.4, 0.4]) + eshift
pbe_shift = np.array([0, -0.3, -0.2, -0.25, -0.35, -0.3, -0.25, -0.2, -0.4, -0.4, -0.4, -0.4, -0.37, -0.4]) + eshift
pbesol_shift = np.array([0, -0.2, -0.1, -0.15, -0.2, -0.05, 0, 0, -0.2, -0.1, -0.1, -0.1, 0, 0]) + eshift
hse_shift = np.array([-0.15, -0.4, -0.2, -0.2, -0.2, -0.15, 0, 0, 0.1, 0, 0.1, 0.1, 0.2, 0.2]) + eshift

In [ ]:
# Plot initializations vs theories
bseg = bcsm[:, :70]

f, ax = plt.subplots(2, 2, figsize=(4.5, 9))
ax[0,0].imshow(bseg, cmap='Blues', extent=[0, 69, elo, ehi], aspect=17)
ax[0,1].imshow(bseg, cmap='Blues', extent=[0, 69, elo, ehi], aspect=17)
ax[1,0].imshow(bseg, cmap='Blues', extent=[0, 69, elo, ehi], aspect=17)
ax[1,1].imshow(bseg, cmap='Blues', extent=[0, 69, elo, ehi], aspect=17)

# Plot original theory energy bands after zero adjustment
ax[0,0].plot(dftbands['LDA'][:69,:14] - lda_th_shift, 'r-')
ax[0,0].plot(dftbands['LDA'][:69, 13] - lda_th_shift, 'r-', label='Theory')
ax[0,1].plot(dftbands['PBE'][:69,:14] - pbe_th_shift, 'r-')
ax[1,0].plot(dftbands['PBEsol'][:69,:14] - pbesol_th_shift, 'r-')
ax[1,1].plot(dftbands['HSE'][:69,:14] - hse_th_shift, 'r-')

# Plot shift-aligned energy bands
for ib in range(14):
    ax[0,0].plot(dftbands['LDA'][:69,ib] + lda_shift[ib], '--', color='#F0FF24', dashes=[5, 2])
    ax[0,1].plot(dftbands['PBE'][:69,ib] + pbe_shift[ib], '--', color='#F0FF24', dashes=[5, 2])
    ax[1,0].plot(dftbands['PBEsol'][:69,ib] + pbesol_shift[ib], '--', color='#F0FF24', dashes=[5, 2])
    ax[1,1].plot(dftbands['HSE'][:69,ib] + hse_shift[ib], '--', color='#F0FF24', dashes=[5, 2])
ax[0,0].plot(dftbands['LDA'][:69,ib] + lda_shift[ib], '--', color='#F0FF24', dashes=[5, 2], label='Initialization')

ax[0,0].set_title('LDA', fontsize=15, x=0.8, y=0.88)
ax[0,1].set_title('PBE', fontsize=15, x=0.8, y=0.88)
ax[1,0].set_title('PBEsol', fontsize=15, x=0.75, y=0.88)
ax[1,1].set_title('HSE06', fontsize=15, x=0.76, y=0.88)

for i in range(2):
    for j in range(2):
        ax[i,j].tick_params(axis='both', length=8, width=2, labelsize=15)
        ax[i,j].set_ylim([elo, ehi])

ax[0,0].set_xticks([])
ax[0,1].set_xticks([])
ax[0,1].set_yticks([])
ax[1,1].set_yticks([])
ax[0,0].set_yticks(range(-7, 1))
ax[1,0].set_yticks(range(-7, 1))

ax[0,0].set_ylabel('Energy (eV)', fontsize=20)
ax[1,0].set_ylabel('Energy (eV)', fontsize=20)

ax[1,0].tick_params(axis='x', width=0, pad=0)
ax[1,1].tick_params(axis='x', width=0, pad=0)

ax[1,0].set_xticks([0,  69])
ax[1,0].set_xticklabels(['$\overline{\Gamma}$', '$\overline{\mathrm{M}}$'])
ax[1,1].set_xticks([0,  69])
ax[1,1].set_xticklabels(['$\overline{\Gamma}$', '$\overline{\mathrm{M}}$'])

lg = ax[0,0].legend(fontsize=15, ncol=2, facecolor='#A9CCE3', edgecolor='#A9CCE3', bbox_to_anchor=(2.21, 1.18))

plt.subplots_adjust(hspace=0.02, wspace=0.15)
plt.savefig('../results/figures/fig_2a.png', bbox_inches='tight', transparent=True, dpi=300)

### Main Figure 2c

In [ ]:
paths = np.load(r'../data/processed/hslines/WSe2_kpath.npz')

reconbands = {}
for name in ['LDA', 'PBE', 'PBEsol', 'HSE']:
    bands = np.load(r'../data/processed/wse2_recon/postproc_refrotsym_bands_'+name.lower()+'.npy')
    bdi = aly.bandpath_map(np.moveaxis(bands, 0, 2), pathr=paths['rowInds'], pathc=paths['colInds'], eaxis=2)
    reconbands[name] = bdi.T

In [ ]:
pos = paths['pathInds']
pos[-1] -= 1

ff, axa = plt.subplots(1, 1, figsize=(10.5, 8))
im = axa.imshow(bcsm, cmap='Blues', extent=[0, 185, elo, ehi], aspect=12)
# axa.plot(dftbands['HSE'][:,:14] - hse_th_shift, 'r--', zorder=2)
for ib in range(14):
    axa.plot(reconbands['LDA'][:,ib] + 0.65, color='r', zorder=1);
axa.tick_params(axis='y', length=8, width=2, labelsize=15)
axa.tick_params(axis='x', length=0, labelsize=15, pad=8)
axa.set_ylim([elo, ehi])
axa.set_xticks(pos)
axa.set_xticklabels(['$\overline{\Gamma}$', '$\overline{\mathrm{M}}$',
                       '$\overline{\mathrm{K}}$', '$\overline{\Gamma}$']);
axa.set_ylabel('Energy (eV)', fontsize=20)
for p in pos[:-1]:
        axa.axvline(x=p, c='k', ls='--', lw=2, dashes=[4, 2])
        
axa.set_title('Reconstruction', fontsize=15, x=0.8, y=0.9)
cax = inset_axes(axa, width="3%", height="30%", bbox_to_anchor=(220, 90, 440, 200))
cb = plt.colorbar(im, cax=cax, ticks=[])
cb.ax.set_ylabel('Intensity', fontsize=15, rotation=-90, labelpad=17)

plt.savefig('../results/figures/fig_2c.png', bbox_inches='tight', transparent=True, dpi=300)

### Supplementary Figure 5

In [ ]:
dftbands = {}
for dft in ['LDA','PBE','PBEsol','HSE']:
    dftbands[dft] = sio.loadmat(r'../data/theory/hslines/WSe2_bandlines_'+dft+'_186.mat')['lines']

In [ ]:
f, axs = plt.subplots(2, 2, figsize=(17, 10))
axs[0,0].imshow(bcsm, cmap='Blues', extent=[0, 185, elo, ehi], aspect=12)
axs[0,1].imshow(bcsm, cmap='Blues', extent=[0, 185, elo, ehi], aspect=12)
axs[1,0].imshow(bcsm, cmap='Blues', extent=[0, 185, elo, ehi], aspect=12)
axs[1,1].imshow(bcsm, cmap='Blues', extent=[0, 185, elo, ehi], aspect=12)

c_over = '#FFD54F'
axs[0,0].plot(dftbands['LDA'][:,:14] - lda_th_shift, color=c_over, ls='--', dashes=[3, 1], zorder=2)
axs[0,1].plot(dftbands['PBE'][:,:14] - pbe_th_shift, color=c_over, ls='--', dashes=[3, 1], zorder=2)
axs[1,0].plot(dftbands['PBEsol'][:,:14] - pbesol_th_shift, color=c_over, ls='--', dashes=[3, 1], zorder=2)
axs[1,1].plot(dftbands['HSE'][:,:14] - hse_th_shift, color=c_over, ls='--', dashes=[3, 1], zorder=2)

axs[0,0].set_title('LDA', fontsize=15, x=0.8, y=0.9)
axs[0,1].set_title('PBE', fontsize=15, x=0.8, y=0.9)
axs[1,0].set_title('PBEsol', fontsize=15, x=0.8, y=0.9)
axs[1,1].set_title('HSE06', fontsize=15, x=0.8, y=0.9)

for i in range(2):
    for j in range(2):
        axs[i,j].tick_params(axis='y', length=8, width=2, labelsize=15)
        axs[i,j].tick_params(axis='x', length=0, labelsize=15, pad=8)
        axs[i,j].set_yticks(np.arange(-8, 1))
        axs[i,j].set_ylim([elo, ehi])
        axs[i,j].set_xticks(pos)
        axs[i,j].set_xticklabels(['$\overline{\Gamma}$', '$\overline{\mathrm{M}}$',
                               '$\overline{\mathrm{K}}$', '$\overline{\Gamma}$'])
    
        for p in pos[:-1]:
            axs[i,j].axvline(x=p, c='k', ls='--', lw=2, dashes=[3, 1.7])
        
for ib in range(14):
    axs[0,0].plot(reconbands['LDA'][:,ib] + 0.65, '-', color='r', zorder=1)
    axs[0,1].plot(reconbands['PBE'][:,ib] + 0.65, '-', color='r', zorder=1)
    axs[1,0].plot(reconbands['PBEsol'][:,ib] + 0.65, '-', color='r', zorder=1)
    axs[1,1].plot(reconbands['HSE'][:,ib] + 0.65, '-', color='r', zorder=1)

# Add legend
axs[0,0].plot(dftbands['LDA'][:,0] - lda_th_shift, color=c_over, ls='--', dashes=[3, 1], zorder=2, label='LDA Calc.')
axs[0,0].plot(reconbands['LDA'][:,ib] + 0.65, '-', color='r', zorder=1, label='LDA Recon.')
lg = axs[0,0].legend(fontsize=15, ncol=1, facecolor='w', edgecolor='k', framealpha=1, bbox_to_anchor=(0.94, 0.1))
lg.get_frame().set_linewidth(2)

axs[0,1].plot(dftbands['PBE'][:,0] - pbe_th_shift, color=c_over, ls='--', dashes=[3, 1], zorder=2, label='PBE Calc.')
axs[0,1].plot(reconbands['PBE'][:,ib] + 0.65, '-', color='r', zorder=1, label='PBE Recon.')
lg = axs[0,1].legend(fontsize=15, ncol=1, facecolor='w', edgecolor='k', framealpha=1, bbox_to_anchor=(0.94, 0.1))
lg.get_frame().set_linewidth(2)

axs[1,0].plot(dftbands['PBEsol'][:,0] - pbesol_th_shift, color=c_over, ls='--', dashes=[3, 1], zorder=2, label='PBEsol Calc.')
axs[1,0].plot(reconbands['PBEsol'][:,ib] + 0.65, '-', color='r', zorder=1, label='PBEsol Recon.')
lg = axs[1,0].legend(fontsize=15, ncol=1, facecolor='w', edgecolor='k', framealpha=1, bbox_to_anchor=(0.98, 0.1))
lg.get_frame().set_linewidth(2)

axs[1,1].plot(dftbands['HSE'][:,0] - hse_th_shift, color=c_over, ls='--', dashes=[3, 1], zorder=2, label='HSE06 Calc.')
axs[1,1].plot(reconbands['HSE'][:,ib] + 0.65, '-', color='r', zorder=1, label='HSE06 Recon.')
lg = axs[1,1].legend(fontsize=15, ncol=1, facecolor='w', edgecolor='k', framealpha=1, bbox_to_anchor=(0.94, 0.1))
lg.get_frame().set_linewidth(2)

plt.subplots_adjust(hspace=0.2, wspace=0.1)
plt.savefig('../results/figures/sfig_5.png', bbox_inches='tight', transparent=True, dpi=300)